# Train Model by country

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from tqdm.auto import tqdm

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# set the font size for matplotlib and the font family.
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

In [2]:
# load the necessary functions from the analysis package

# load the variable names, this allows to access the variables in the feature data in a compact way
from analysis_utils.variable_names import *

# load flagged ids 
from analysis_utils.flagged_uids import *

# load the functions to do spatial k-fold CV
from analysis_utils.spatial_CV import *

# load the helper functions
from analysis_utils.analysis_helpers import *

# load the random forest trainer and cross_validator
import analysis_utils.RandomForest as rf

# load the combien model
from analysis_utils.CombinedModel import CombinedModel

In [3]:
# set the global file paths
root_data_dir = "../../Data"

# the lsms data
lsms_pth = f"{root_data_dir}/lsms/processed/labels_cluster_v1.csv"

# the feature data
feat_data_pth = f"{root_data_dir}/feature_data/tabular_data.csv"

# set the random seed
random_seed = 423
spatial_cv_random_seed = 348

# set the number of folds for k-fold CV
n_folds = 5

In [4]:
# load the feature and the label data
lsms_df = pd.read_csv(lsms_pth)

# remove flagged ids form dataset
lsms_df = lsms_df[~lsms_df.unique_id.isin(flagged_uids)].reset_index(drop = True)

# create average varaibles for the between model
lsms_df['avg_log_mean_pc_cons_usd_2017'] = lsms_df.groupby('cluster_id')['log_mean_pc_cons_usd_2017'].transform('mean')
lsms_df['avg_mean_asset_index_yeh'] = lsms_df.groupby('cluster_id')['mean_asset_index_yeh'].transform('mean')

# load the feature data
feat_df = pd.read_csv(feat_data_pth)

# describe the training data broadly
print(f"Number of observations {len(lsms_df)}")
print(f"Number of clusters {len(np.unique(lsms_df.cluster_id))}")
print(f"Number of x vars {len(feat_df.columns)-2}")

Number of observations 6401
Number of clusters 2128
Number of x vars 113


In [5]:
# merge the label and the feature data to one dataset
lsms_vars = ['unique_id','n_households', 'country', 'lat', 'lon',        
             'log_mean_pc_cons_usd_2017', 'avg_log_mean_pc_cons_usd_2017',
             'mean_asset_index_yeh', 'avg_mean_asset_index_yeh']
df = pd.merge(lsms_df[lsms_vars], feat_df, on = 'unique_id', how = 'left')

In [6]:
# define the x variables and the target variable for the between and within models

# define the between variables
avg_rs_vars = avg_ndvi_vars + avg_ndwi_gao_vars + avg_nl_vars
osm_vars = osm_dist_vars + osm_count_vars + osm_road_vars
between_x_vars = osm_vars + esa_lc_vars + wsf_vars + avg_rs_vars + avg_preciptiation + median_rgb_vars 
between_target_var = 'avg_log_mean_pc_cons_usd_2017'

# define the within varaibles
dyn_rs_vars = dyn_ndvi_vars + dyn_ndwi_gao_vars + dyn_nl_vars
within_x_vars = dyn_rs_vars + precipitation + dyn_rgb_vars
within_target_var = 'log_mean_pc_cons_usd_2017'

In [7]:
# define the between and the within df

between_df = df[['cluster_id', 'country', between_target_var] + between_x_vars].drop_duplicates().reset_index(drop = True)
within_df = df[['unique_id', 'cluster_id', 'country',within_target_var] + within_x_vars]

# Run the CV

In [8]:
# train a model on each country and evaluate on the remaining countries.
# for the country itself, use spatial cross validation to evaluate the model

In [9]:
countries = ['eth', 'mwi', 'nga', 'tza', 'uga']
cross_cntry_res = {}

between_df_norm = standardise_df(between_df, exclude_cols = [between_target_var])
demeaned_df_norm = standardise_df(demean_df(within_df), exclude_cols = [within_target_var])
demeaned_df_norm['country'] = within_df['country']

for cntry in tqdm(countries):
    
    # subset the training data
    train_between_df_norm = between_df_norm[between_df_norm.country == cntry].reset_index(drop = True)
    train_demeaned_df_norm = demeaned_df_norm[demeaned_df_norm.country == cntry].reset_index(drop = True)
    
    # get the X and y vectors
    X_train_between = train_between_df_norm.loc[:, between_x_vars]
    y_train_between = np.array(train_between_df_norm[[between_target_var]]).ravel()
    
    X_train_within = train_demeaned_df_norm.loc[:, within_x_vars]
    y_train_within = np.array(train_demeaned_df_norm[[within_target_var]]).ravel()
    
    # train the between model
    between_rf = rf.Trainer(X_train_between, y_train_between, None, None, random_seed)
    between_rf.train(min_samples_leaf = 5)
    
    # train within model
    within_rf = rf.Trainer(X_train_within, y_train_within, None, None, random_seed)
    within_rf.train(min_samples_leaf = 15)
    
    # evaluate the between and the within models on all other countries
    val_countries = [i for i in countries if i != cntry]
    
    # define object to store results of country
    cross_cntry_res[cntry] = {
        'between_r2': [],
        'within_r2': [], 
        'overall_r2': [],
        'val_cntry': []
    }
    
    for val_cntry in val_countries:
        
        # subset the data to the validation country
        val_df = df.loc[df.country == val_cntry].reset_index(drop = True)
        
        # normalise data
        val_between_df_norm = between_df_norm[between_df_norm.country == val_cntry].reset_index(drop = True)
        val_demeaned_df_norm = demeaned_df_norm[demeaned_df_norm.country == val_cntry].reset_index(drop = True)
    
        # get the X and y vectors
        X_val_between = val_between_df_norm.loc[:, between_x_vars]
        y_val_between = np.array(val_between_df_norm[[between_target_var]]).ravel()

        X_val_within = val_demeaned_df_norm.loc[:, within_x_vars]
        y_val_within = np.array(val_demeaned_df_norm[[within_target_var]]).ravel()
        
        # validate both models on the validation country
        between_preds, between_r2, between_mse = between_rf.validate(X_val_between, y_val_between)
        between_preds_df = pd.DataFrame({
            'cluster_id': val_between_df_norm.cluster_id,
            'between_y_hat': between_preds
        })
        
        # validate the within model
        within_preds, within_r2, within_mse = within_rf.validate(X_val_within, y_val_within)
        within_preds_df = pd.DataFrame({
            'unique_id': val_demeaned_df_norm.unique_id,
            'cluster_id': val_demeaned_df_norm.cluster_id,
            'within_y_hat': within_preds
        })
        
        # combine both predictions
        preds_df = pd.merge(within_preds_df, between_preds_df, on = 'cluster_id', how = 'left')
        preds_df['y_hat'] = preds_df['between_y_hat'] + preds_df['within_y_hat']
        preds_df['y'] = val_df['log_mean_pc_cons_usd_2017']
        
        # evaluate the predictions
        overall_r2 = r2_score(preds_df['y'], preds_df['y_hat'])
        
        # results
        cross_cntry_res[cntry]['between_r2'].append(between_r2)
        cross_cntry_res[cntry]['within_r2'].append(within_r2)
        cross_cntry_res[cntry]['overall_r2'].append(overall_r2)
        cross_cntry_res[cntry]['val_cntry'].append(val_cntry)

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
# save results
pth = 'results/robustness/cross_cntry_res.pkl'
with open(pth, 'wb') as f:
    pickle.dump(cross_cntry_res, f)

# Train on country and evaluate on country

In [13]:
same_cntry_res = {
    'country': [],
    'between_r2': [],
    'within_r2': [],
    'overall_r2': []
}
it = 0
for cntry in tqdm(countries):
    print("="*100)
    print(f"Training and Evaluating on country {cntry}")
    print("="*100)
    print("\n")
    
    # subset the df to country
    df_sub = df.loc[df.country == cntry].reset_index(drop = True)
    between_df_sub = between_df[between_df.country == cntry].reset_index(drop = True)
    within_df_sub = within_df[within_df.country == cntry].reset_index(drop = True)
    
    # standardise the datasets
    between_df_norm = standardise_df(between_df_sub, exclude_cols = [between_target_var])
    demeaned_df_norm = standardise_df(demean_df(within_df_sub), exclude_cols = [within_target_var])

    aux_r2_b = []
    aux_r2_w = []
    aux_r2_o = []
    
    for j in range(10):
        print("."*100)
        print(f"Iteration {j}")
        print("."*100)
        
        seeed = random_seed + j + it
        
        # get training and validation ids using spatial CV scheme
        fold_ids = split_lsms_spatial(df_sub, n_folds = n_folds, random_seed = seeed)
        
        # run the bewtween training
        print('\n\tBetween model')
        between_cv_trainer = rf.CrossValidator(between_df_norm, 
                                               fold_ids,
                                               between_target_var, 
                                               between_x_vars, 
                                               id_var = 'cluster_id',
                                               random_seed = seeed)
        between_cv_trainer.run_cv_training(min_samples_leaf = 5)
        between_res = between_cv_trainer.compute_overall_performance()

        # run the within training
        print('\n\tWithin Model')
        within_cv_trainer = rf.CrossValidator(demeaned_df_norm, 
                                              fold_ids, 
                                              within_target_var, 
                                              within_x_vars,
                                              id_var = 'unique_id',
                                              random_seed = seeed)
        within_cv_trainer.run_cv_training(min_samples_leaf = 15)
        within_res = within_cv_trainer.compute_overall_performance()

        # run the combined model
        combined_model = CombinedModel(df_sub, between_cv_trainer, within_cv_trainer)
        combined_model.evaluate()
        combined_res = combined_model.compute_overall_performance()
        print(f"\t{combined_res['r2']}")

        # store results
        aux_r2_b.append(combined_res['r2']['between'])
        aux_r2_w.append(combined_res['r2']['within'])
        aux_r2_o.append(combined_res['r2']['overall'])
        
    it += 1
    
    # take the average over all results
    same_cntry_res['country'].append(cntry)
    same_cntry_res['between_r2'].append(np.mean(aux_r2_b))
    same_cntry_res['within_r2'].append(np.mean(aux_r2_w))
    same_cntry_res['overall_r2'].append(np.mean(aux_r2_o))

  0%|          | 0/5 [00:00<?, ?it/s]

Training and Evaluating on country eth


....................................................................................................
Iteration 0
....................................................................................................
Fold 0, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 1, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 2, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 3, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 4, specified test ratio: 0.2 - Actual test ratio 0.19

	Between model
Initialising training


  0%|          | 0/5 [00:00<?, ?it/s]

Finished training after 53 seconds

	Within Model
Initialising training


  0%|          | 0/5 [00:00<?, ?it/s]

...

Finished training after 79 seconds
	{'overall': 0.2951623370670927, 'between': 0.33545632111360973, 'within': 0.05096173493981095}
....................................................................................................
Iteration 9
....................................................................................................
Fold 0, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 1, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 2, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 3, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 4, specified test ratio: 0.2 - Actual test ratio 0.20

	Between model
Initialising training


  0%|          | 0/5 [00:00<?, ?it/s]

Finished training after 40 seconds

	Within Model
Initialising training


  0%|          | 0/5 [00:00<?, ?it/s]

Finished training after 79 seconds
	{'overall': 0.1763781938773185, 'between': 0.1957841283104348, 'within': 0.05518397683346445}


In [14]:
pth = 'results/robustness/same_cntry_res.pkl'
with open(pth, 'wb') as f:
    pickle.dump(same_cntry_res, f)